In [1]:
import xarray as xr
import numpy as np
import os, sys
import psycopg2, psycopg2.extras
import pandas as pd
from PIL import Image

# adds the package path to the Python path to make sure all the local imports work fine 
if os.path.dirname(os.path.dirname(os.getcwd())) not in sys.path:
    sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
    
from flares_package.constants import DB_HOST, DB_NAME, DB_USER, DB_PASS, DATA_DIR_DEM, DATA_DIR_LC

### Create Xarray dataset with the longitude and latitude dimenstions, to store the information in

In [2]:
lat_values = np.arange(55.65, 51.25, -0.1)
lon_values = np.arange(348.65, 354.75, 0.1)

grid = xr.Dataset(coords={'longitude':lon_values, 'latitude': lat_values})

## Combine resampled DEM & Slope rasters into xarray dataset

In [4]:
# DEM 

corine = Image.open(f"{DATA_DIR_LC}/ireland/corine_2018_resampled.tif")
corine_ar = xr.DataArray(np.array(corine), [("latitude", lat_values), ("longitude", lon_values)])

grid['corine'] = corine_ar 

dem_avg = Image.open(f"{DATA_DIR_DEM}/dem_ireland_4326_resampled_average.tif")
dem_avg_ar = xr.DataArray(np.array(dem_avg), [("latitude", lat_values), ("longitude", lon_values)])

grid['elevation_avg'] = dem_avg_ar

dem_med = Image.open(f"{DATA_DIR_DEM}/dem_ireland_4326_resampled_median.tif")
dem_med_ar = xr.DataArray(np.array(dem_med), [("latitude", lat_values), ("longitude", lon_values)])

grid['elevation_med'] = dem_med_ar

dem_q1 = Image.open(f"{DATA_DIR_DEM}/dem_ireland_4326_resampled_Q1.tif")
dem_q1_ar = xr.DataArray(np.array(dem_q1), [("latitude", lat_values), ("longitude", lon_values)])

grid['elevation_q1'] = dem_q1_ar

dem_q3 = Image.open(f"{DATA_DIR_DEM}/dem_ireland_4326_resampled_Q3.tif")
dem_q3_ar = xr.DataArray(np.array(dem_q3), [("latitude", lat_values), ("longitude", lon_values)])

grid['elevation_q3'] = dem_q3_ar

In [5]:
# Slope

slope_avg = Image.open(f"{DATA_DIR_DEM}/slope_ireland_4326_resampled_average.tif")
slope_avg_ar = xr.DataArray(np.array(slope_avg), [("latitude", lat_values), ("longitude", lon_values)])

grid['slope_avg'] = slope_avg_ar

slope_med = Image.open(f"{DATA_DIR_DEM}/slope_ireland_4326_resampled_median.tif")
slope_med_ar = xr.DataArray(np.array(slope_med), [("latitude", lat_values), ("longitude", lon_values)])

grid['slope_med'] = slope_med_ar

slope_q1 = Image.open(f"{DATA_DIR_DEM}/slope_ireland_4326_resampled_Q1.tif")
slope_q1_ar = xr.DataArray(np.array(slope_q1), [("latitude", lat_values), ("longitude", lon_values)])

grid['slope_q1'] = slope_q1_ar

slope_q3 = Image.open(f"{DATA_DIR_DEM}/slope_ireland_4326_resampled_Q3.tif")
slope_q3_ar = xr.DataArray(np.array(slope_q3), [("latitude", lat_values), ("longitude", lon_values)])

grid['slope_q3'] = slope_q3_ar

## Create Database Table

In [ ]:
conn = psycopg2.connect(dbname=DB_NAME, user=DB_USER, password=DB_PASS, host=DB_HOST)
cur = conn.cursor()

# create table
# column - Geometry, Date/Time, Concentration

cur.execute(
    f"""
    CREATE TABLE spatial_baseline_info(
    ID SERIAL PRIMARY KEY,
    GEOMETRY geometry,
    CORINE_GRID_CODE SMALLINT,
    CORINE_LC_CODE SMALLINT,
    CORINE_LABEL_1 varchar(300),
    CORINE_LABEL_2 varchar(300),
    CORINE_LABEL_3 varchar(300),
    ELEVATION_AVG numeric,
    ELEVATION_MED numeric,
    ELEVATION_Q1 numeric,
    ELEVATION_Q3 numeric,
    SLOPE_AVG numeric,
    SLOPE_MED numeric, 
    SLOPE_Q1 numeric, 
    SLOPE_Q3 numeric 
    )"""
    )

conn.commit()

## Add information to database table

In [ ]:
df_corine_leg = pd.read_csv(f"{DATA_DIR_LC}\Legend\clc_legend.csv").dropna()

conn = psycopg2.connect(dbname=DB_NAME, user=DB_USER, password=DB_PASS, host=DB_HOST)
cur = conn.cursor()

cur.execute("BEGIN")

for lat in lat_values:
    for lon in lon_values:
        
        grid_cell = grid.sel(latitude = lat, longitude=lon)
        corine_grid_code = int(grid_cell['corine'].values)
        
        lc_class_metada = df_corine_leg[df_corine_leg['GRID_CODE'].astype(int) == int(corine_grid_code)]
        
        corine_lc_code = lc_class_metada['CLC_CODE']
        corine_label_1 = lc_class_metada['LABEL1'].iloc[0]
        corine_label_2 = lc_class_metada['LABEL2'].iloc[0]
        corine_label_3 = lc_class_metada['LABEL3'].iloc[0]
        elevation_avg = grid_cell['elevation_avg'].values
        elevation_med = grid_cell['elevation_med'].values
        elevation_q1 = grid_cell['elevation_q1'].values
        elevation_q3 = grid_cell['elevation_q3'].values
        slope_avg = grid_cell['slope_avg'].values
        slope_med = grid_cell['slope_med'].values
        slope_q1 = grid_cell['slope_q1'].values
        slope_q3 = grid_cell['slope_q3'].values
        
        values = (
            round(float(lon - 360), 2),
            round(float(lat), 2),
            int(corine_grid_code),
            int(corine_lc_code),
            corine_label_1,
            corine_label_2,
            corine_label_3,
            round(float(elevation_avg), 2),
            round(float(elevation_med), 2),
            round(float(elevation_q1), 2),
            round(float(elevation_q3), 2),
            round(float(slope_avg), 2),
            round(float(slope_med), 2),
            round(float(slope_q1), 2),
            round(float(slope_q3), 2),
        )
        
        cur.execute(f"""
            INSERT INTO spatial_baseline_info (
                GEOMETRY,
                CORINE_GRID_CODE,
                CORINE_LC_CODE,
                CORINE_LABEL_1,
                CORINE_LABEL_2,
                CORINE_LABEL_3,
                ELEVATION_AVG,
                ELEVATION_MED,
                ELEVATION_Q1,
                ELEVATION_Q3,
                SLOPE_AVG,
                SLOPE_MED,
                SLOPE_Q1,
                SLOPE_Q3
                ) VALUES (ST_SetSRID(ST_MakePoint(%s,%s), 4326), %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)""", values)
        conn.commit()